In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parser
import json
import time

In [2]:
name = '쯔양' #input()

In [3]:
driver = webdriver.Chrome('./chromedriver.exe')
url = 'https://some.co.kr/analysis/social/reputation'
driver.get(url)

In [4]:
# 팝업창 닫기
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[13]/div[5]/div/p/a"))).click()

In [5]:
# 모달 열기
driver.find_element_by_xpath("//button[@type='button']").click()

In [6]:
# 검색 시작
find = driver.find_element_by_xpath("/html/body/div[4]/div/div[1]/div[2]/div[1]/form/fieldset/div/div[1]/div/input")
time.sleep(2)
find.send_keys(name)
driver.find_element_by_xpath("/html/body/div[4]/div/div[1]/div[2]/div[2]/div/div[2]/button").click()
time.sleep(4)

In [7]:
html = driver.page_source
driver.quit()
soup = BeautifulSoup(html, 'html.parser')

In [8]:
# 댓글 평판 크롤링
temp = soup.find_all('table')
p = parser.make2d(temp[1])

In [9]:
positive = [ [x[0],int(x[2])] for x in p if x[1]=='긍정']
negative = [ [x[0],int(x[2])] for x in p if x[1]=='부정']
neutral = [ [x[0],int(x[2])] for x in p if x[1]=='중립']

In [10]:
# 전체적인 분위기
p2 = list((soup.find('span',id='rateOfSentiment').text).split())
p2

['긍정', '73%']

In [11]:
# 긍정, 부정, 중립 건수
p3 = [int(x.text[:-1]) for x in soup.find_all('span','legend-value')]
p3

[409, 115, 35]

In [12]:
p3_rate = [ [x, x/sum(p3)*100] for x in p3]
p3_rate

[[409, 73.16636851520573], [115, 20.572450805008945], [35, 6.261180679785331]]

In [13]:
data = {}
data['name'] = name
data["rateOfSentiment"]=[]
data["rateOfSentiment"].append({
    "type":"positive",
    "count":p3_rate[0][0],
    "rate":p3_rate[0][1]
})
data["rateOfSentiment"].append({
    "type":"negative",
    "count":p3_rate[1][0],
    "rate":p3_rate[1][1]
})
data["rateOfSentiment"].append({
    "type":"neutral",
    "count":p3_rate[2][0],
    "rate":p3_rate[2][1]
})

In [14]:
data['reputation'] = []
data["reputation"].append({
    "type" : "positive",
    "data" : positive
})
data["reputation"].append({
    "type" : "negative",
    "data" : negative
})
data["reputation"].append({
    "type" : "neutral",
    "data" : neutral
})

In [15]:
data

{'name': '쯔양',
 'rateOfSentiment': [{'type': 'positive',
   'count': 409,
   'rate': 73.16636851520573},
  {'type': 'negative', 'count': 115, 'rate': 20.572450805008945},
  {'type': 'neutral', 'count': 35, 'rate': 6.261180679785331}],
 'reputation': [{'type': 'positive',
   'data': [['이쁘다', 29],
    ['응원하다', 29],
    ['맛있다', 27],
    ['먹고싶다', 24],
    ['좋다', 18],
    ['유명하다', 16],
    ['좋아하다', 15],
    ['즐기다', 15],
    ['명성 자자하다', 12],
    ['감칠맛', 11],
    ['손맛', 11],
    ['웃음', 11],
    ['진심', 10],
    ['잘먹다', 10],
    ['흥행', 10],
    ['감사드리다', 8],
    ['신기하다', 7],
    ['가고싶다', 5],
    ['웃다', 5],
    ['신선하다', 5],
    ['손님 많다', 4],
    ['재미있다', 4],
    ['부럽다', 4],
    ['괜찮다', 3]]},
  {'type': 'negative',
   'data': [['죄송하다', 31], ['정 떨어지다', 24], ['정떨어지다', 5], ['심심하다', 3]]},
  {'type': 'neutral', 'data': [['달다', 14], ['배부르다', 4]]}]}

In [16]:
with open("./sometrend.json", 'w',encoding="UTF-8") as outfile:
    json.dump(data, outfile,ensure_ascii = False)